In [1]:
HOPSWORKS_PROJECT_NAME = "taxi_demand_jithis"
%reload_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os
from pathlib import Path
from src.paths import PARENT_DIR

# Ensure PARENT_DIR is a Path object, not a string
parent_dir_path = Path(PARENT_DIR)

# Load environment variables from .env file
# Make sure the path is constructed correctly
dotenv_path = parent_dir_path / '.env'
load_dotenv(dotenv_path)

# Access API key with error handling
HOPSWORKS_API_KEY= os.getenv('HOPSWORKS_API_KEY')


In [3]:


from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])



File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
2023-11 file is not available
2023-12 file is not available


In [4]:


print(f'{len(rides)=:,}')



len(rides)=71,247,922


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)


100%|██████████| 265/265 [00:02<00:00, 122.78it/s]


In [11]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01
1,2022-01-01 00:33:43,236,2022-01-01
2,2022-01-01 00:53:21,166,2022-01-01
3,2022-01-01 00:25:21,114,2022-01-01
4,2022-01-01 00:36:48,68,2022-01-01


In [15]:
ts_data.head(-35)

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
4254800,2023-10-30 08:00:00,1,265
4254801,2023-10-30 09:00:00,2,265
4254802,2023-10-30 10:00:00,3,265
4254803,2023-10-30 11:00:00,7,265


In [6]:
import hopsworks

In [7]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)


Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/325782


In [8]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [9]:


FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1



In [16]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [17]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/325782/fs/324677/fg/346470


Uploading Dataframe: 0.00% |          | Rows 0/4254840 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/325782/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f1ff63851b0>, None)